# Linear Algebra

After `using LinearAlgebra`, **Julia speaks linear algebra fluently**.

Performing linear algebra operations on a computer is, of course, an old problem. Lots of amazing libraries have been written - mostly in Fortran - which have been optimized over decades.

Basically all high-level programming languages use these libraries, including R, Python, and Julia.

Linear algebra in Julia is largely implemented by calling [BLAS](http://www.netlib.org/blas/)/[LAPACK](http://www.netlib.org/lapack/) functions. Sparse operations utilize functionality in [SuiteSparse](http://faculty.cse.tamu.edu/davis/suitesparse.html).

As per default, Julia uses the [OpenBLAS](https://github.com/xianyi/OpenBLAS) implementation (BLAS, LAPACK, LIBM), which can be replaced by [Intel's MKL](https://software.intel.com/en-us/mkl) (BLAS, LAPACK) and [Intel's Math Library](https://software.intel.com/en-us/node/522653) (LIBM).

**What is all this stuff?!?**

* **BLAS**: a collection of low-level matrix and vector arithmetic operations ("multiply two matrices", "multiply a matrix by vector").
* **LAPACK**:  a collection of higher-level linear algebra operations. Things like matrix factorizations (LU, LLt, QR, SVD, Schur, etc) that are used to do things like “find the eigenvalues of a matrix”, or “find the singular values of a matrix”, or “solve a linear system”.
* **LIBM**: basic math functions like `sin`, `cos`, `sinh`, etcetera

Sparse matrices are more difficult and there exist different collections of routines, one of which is **SuiteSparse**.

**Why do I have to care?**

* Switching from OpenBLAS to MKL can give you large speedups!
* Since you might be leaving the world of Julia code, you loose easy inspectability and type genericity. The latter can be an issue for machine learning, as we'll discuss later in more detail.

# Taking linear algebra seriously

Julia is [taking linear algebra seriously](https://www.youtube.com/watch?v=C2RO34b_oPM)! (see [here](https://github.com/JuliaLang/julia/issues/4774), and [here](https://github.com/JuliaLang/julia/issues/20978)).

In [1]:
using LinearAlgebra

In [2]:
A = rand(4,4)

4×4 Array{Float64,2}:
 0.312523  0.841416  0.589068   0.941542
 0.113238  0.284005  0.589186   0.670919
 0.21455   0.68334   0.0330208  0.868224
 0.669114  0.932694  0.887386   0.722942

In [3]:
typeof(A)

Array{Float64,2}

In [4]:
Array{Float64, 2} === Matrix{Float64} # equivalent not just equal

true

In [5]:
det(A)

-0.04758149937133032

In [6]:
inv(A)

4×4 Array{Float64,2}:
 -7.56996   2.09682    3.67784   3.49607
  5.86921  -3.69846   -2.38118  -1.3519
  1.83531   0.368315  -2.05896  -0.259349
 -2.81855   2.37873    2.19536   0.209956

In [7]:
rank(A)

4

Let's get a vector as well

In [8]:
v = rand(4)

4-element Array{Float64,1}:
 0.1630923622842968
 0.9931388457640886
 0.13961529011608964
 0.7643990719440354

In [9]:
typeof(v)

Array{Float64,1}

In [10]:
Array{Float64,1} === Vector{Float64}

true

In [11]:
norm(v)

1.271503934726197

In [12]:
v^2 # can't square a vector

LoadError: MethodError: no method matching ^(::Array{Float64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float32, ::Integer) at math.jl:907
  ^(!Matched::Float16, ::Integer) at math.jl:915
  ^(!Matched::Regex, ::Integer) at regex.jl:729
  ...

In [13]:
v.^2

4-element Array{Float64,1}:
 0.02659911863547232
 0.9863247669656262
 0.019492429234199877
 0.5843059411889027

Some things might be suprising

In [14]:
1/v

1×4 Transpose{Float64,Array{Float64,1}}:
 0.100878  0.614292  0.086357  0.472808

But if it works, there is typically meaning to it. In this case it is calculating the [Moore-Penrose-Pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse#Vectors) (`transpose(v)/sum(abs2,v)`).

### Identity matrix: `UniformScaling` operator

In [15]:
A + 3

LoadError: MethodError: no method matching +(::Array{Float64,2}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:538
  +(!Matched::Complex{Bool}, ::Real) at complex.jl:301
  +(!Matched::Missing, ::Number) at missing.jl:115
  ...

In [16]:
A .+ 3

4×4 Array{Float64,2}:
 3.31252  3.84142  3.58907  3.94154
 3.11324  3.284    3.58919  3.67092
 3.21455  3.68334  3.03302  3.86822
 3.66911  3.93269  3.88739  3.72294

The `UniformScaling` operator **represents an identity matrix of any size** and is another great example of **duck typing**. It automatically gets loaded into scope when you do `using LinearAlgebra` and has the name `I`.

In [17]:
I

UniformScaling{Bool}
true*I

Although it never actually materializes a full identity matrix it behaves like one.

In [18]:
A + 3I

4×4 Array{Float64,2}:
 3.31252   0.841416  0.589068  0.941542
 0.113238  3.284     0.589186  0.670919
 0.21455   0.68334   3.03302   0.868224
 0.669114  0.932694  0.887386  3.72294

In [19]:
I * A == A

true

Hence, we can calculate things like, say, `A-b*I` without ever allocating a dense identity matrix, which would take up $\mathcal{O}(n^2)$ memory.

Let's benchmark the performance difference!

In [20]:
fullI = Matrix{Float64}(I, 4,4) # alternatively but slower, diagm(ones(4))

4×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [21]:
fast(A) = A + 3*I

fast (generic function with 1 method)

In [22]:
slow(A, fullI) = A + 3*fullI

slow (generic function with 1 method)

In [24]:
function slower(A)
    fullI = Matrix(1.0I, size(A)...)
    A + 3*fullI
end

slower (generic function with 1 method)

In [25]:
using BenchmarkTools
@btime fast($A);
@btime slow($A, $fullI);
@btime slower($A);

  36.282 ns (1 allocation: 208 bytes)
  124.563 ns (2 allocations: 416 bytes)
  170.874 ns (3 allocations: 624 bytes)


# Fast linear algebra with multiple dispatch

Utilizing the structure of matrices by typing them appropriately!

In [26]:
D = Diagonal(1:5)

5×5 Diagonal{Int64,UnitRange{Int64}}:
 1  ⋅  ⋅  ⋅  ⋅
 ⋅  2  ⋅  ⋅  ⋅
 ⋅  ⋅  3  ⋅  ⋅
 ⋅  ⋅  ⋅  4  ⋅
 ⋅  ⋅  ⋅  ⋅  5

In [27]:
Ddense = Matrix(D) # same matrix but type doesn't indicate diagonal structure

5×5 Array{Int64,2}:
 1  0  0  0  0
 0  2  0  0  0
 0  0  3  0  0
 0  0  0  4  0
 0  0  0  0  5

In [28]:
b = rand(5)

5-element Array{Float64,1}:
 0.9193711068495507
 0.6137210582245434
 0.7900045596233616
 0.5025947637095955
 0.09751130085681292

In [29]:
@btime $D*$b
@btime $Ddense*$b

  40.006 ns (1 allocation: 128 bytes)
  61.197 ns (1 allocation: 128 bytes)


5-element Array{Float64,1}:
 0.9193711068495507
 1.2274421164490867
 2.3700136788700847
 2.010379054838382
 0.4875565042840646

What method does it dispatch to?

In [31]:
@edit D*b

In [ ]:
@which Ddense*b

**Dense Diagonal** (`Ddense*b`)

```julia
function (*)(A::AbstractMatrix{T}, x::AbstractVector{S}) where {T,S}
    TS = promote_op(matprod, T, S)
    mul!(similar(x,TS,axes(A,1)),A,x)
end
```

**Diagonal** (`D*b`)
```julia
(*)(D::Diagonal, V::AbstractVector) = D.diag .* V
```

There are a number of [special matrix](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Special-matrices) types are available out-of-the-box.

# Fermions hopping on a chain

$$\mathcal{H} = -t\sum_{\langle i,j \rangle} c_i^\dagger c_j + \mu \sum_i n_i$$

Here, $t$ is the hopping amplitude, $\mu$ is the chemical potential, and $c, c^\dagger$ are creation and annihilation operators.

For simplicity, we'll consider **open boundary conditions** (not periodic), in which case the Hamiltonian is tridiagonal.

Since the fermions are *not* interacting, we can work in the *single particle basis* and do not have to worry about how to construct a basis for the many-body Fock space.

We use the canonical cartesian basis in which one uses $0$s to indicate empty sites and a $1$ for the particle's site, i.e. $|00100\rangle$ represents the basis state which has the particle exclusively on the 3rd site.

If you aren't familiar with second quantization just think of $\mathcal{H}$ as any quantum mechanical operator that can be represented as a matrix.

In [32]:
N = 100 # number of sites
t = 1
μ = -0.5

H = diagm(0 => fill(μ, N), 1 => fill(-t, N-1), -1 => fill(-t, N-1))

100×100 Array{Float64,2}:
 -0.5  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0  -0.5  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0  -0.5  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0  -0.5  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0  -0.5  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0  -0.5  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0

In [33]:
ψ = normalize(rand(N)) # some state

100-element Array{Float64,1}:
 0.042589316601678695
 0.03330774192938872
 0.009779874983957828
 0.16492997411645743
 0.004686019499977374
 0.15795631842352365
 0.10019652926588493
 0.14077288967992102
 0.01771454280483669
 0.025973942899940403
 0.11682447201709203
 0.15228444517118364
 0.04642266191682645
 ⋮
 0.062458164561997154
 0.12173039757223071
 0.13347043834013714
 0.14151938911825898
 0.15085703295216799
 0.06136230782952981
 0.042467472234412516
 0.10415986317742634
 0.04010746307291547
 0.11024283214996149
 0.026951256550257388
 0.0836609815642866

Quantum mechanical expectation value

In [34]:
ev(H, ψ) = ψ'*H*ψ # <φ|H|φ>

ev (generic function with 1 method)

In [35]:
ev(H, ψ)

-2.0208015208536705

In [36]:
@btime ev($H, $ψ);

  1.702 μs (1 allocation: 896 bytes)


In [37]:
typeof(H)

Array{Float64,2}

As long as the code is generic (respects the informal `AbstractArray` interface), we can use the same piece of code for completely different array types.

Let's utilize the sparsity of `H` by indicating it through a type.

In [38]:
using SparseArrays
Hsparse = sparse(H)

100×100 SparseMatrixCSC{Float64,Int64} with 298 stored entries:
  [1  ,   1]  =  -0.5
  [2  ,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [2  ,   2]  =  -0.5
  [3  ,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [3  ,   3]  =  -0.5
  [4  ,   3]  =  -1.0
  [3  ,   4]  =  -1.0
  [4  ,   4]  =  -0.5
  [5  ,   4]  =  -1.0
  [4  ,   5]  =  -1.0
  ⋮
  [96 ,  96]  =  -0.5
  [97 ,  96]  =  -1.0
  [96 ,  97]  =  -1.0
  [97 ,  97]  =  -0.5
  [98 ,  97]  =  -1.0
  [97 ,  98]  =  -1.0
  [98 ,  98]  =  -0.5
  [99 ,  98]  =  -1.0
  [98 ,  99]  =  -1.0
  [99 ,  99]  =  -0.5
  [100,  99]  =  -1.0
  [99 , 100]  =  -1.0
  [100, 100]  =  -0.5

In [39]:
@btime ev($Hsparse, $ψ);

  698.227 ns (1 allocation: 896 bytes)


That's a solid **30x speedup**!

Our `H` isn't just sparse, but actually tridiagonal. Let's try to exploit that.

In [40]:
Htri = Tridiagonal(H)

100×100 Tridiagonal{Float64,Array{Float64,1}}:
 -0.5  -1.0    ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
 -1.0  -0.5  -1.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅   -1.0  -0.5  -1.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅   -1.0  -0.5  -1.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅   -1.0  -0.5  -1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅   -1.0  -0.5  …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅   -1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅    

In [41]:
@btime ev($Htri, $ψ);

  172.283 ns (2 allocations: 944 bytes)


In [45]:
@which D*b

*(D::Diagonal, V::AbstractArray{T,1} where T) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/diagonal.jl:178

Choosing the best type (and therewith an algorithm) can be tricky and one has to play around a bit. The good thing is that it's very easy to try out different types!

Note that there are also great matrix types available in the ecosystem, see [JuliaMatrices](https://github.com/JuliaMatrices), for example.

# Exact diagonalisation a.k.a Eigendecomposition

To diagonalize our dense "Hamiltonian", we simply call the built-in function `eigen`.

In [46]:
vals, vecs = eigen(H)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
100-element Array{Float64,1}:
 -2.4990325645839753
 -2.4961311942671887
 -2.4912986959380365
 -2.484539744726553
 -2.475860879481513
 -2.4652704964445267
 -2.4527788411272136
 -2.438397998399332
 -2.422141880797449
 -2.4040262150654583
 -2.384068526939977
 -2.3622881241953175
 -2.3387060779644715
  ⋮
  1.362288124195319
  1.3840685269399784
  1.4040262150654599
  1.422141880797449
  1.4383979983993322
  1.452778841127214
  1.4652704964445273
  1.4758608794815133
  1.484539744726553
  1.4912986959380372
  1.4961311942671887
  1.4990325645839762
vectors:
100×100 Array{Float64,2}:
 0.00437636  -0.00874848  0.0131121  -0.0174631  …   0.00874848  -0.00437636
 0.00874848  -0.0174631   0.0261102  -0.0346562     -0.0174631    0.00874848
 0.0131121   -0.0261102   0.038881   -0.0513136      0.0261102   -0.0131121
 0.0174631   -0.0346562   0.0513136  -0.0671776     -0.0346562    0.0174631
 0.0217972   -0.0430682   0.0632996  -0.0820

Since Julia is using eigenproblem solvers from LAPACK (written in a low-level language) the code is, of course, **not generic**.

The best Julia can do, without implementing new functionality, is manually dispatch to the best LAPACK routine available.

Hence, it won't work with most of our special matrices.

In [47]:
eigen(Htri);

LoadError: MethodError: no method matching eigen!(::Tridiagonal{Float64,Array{Float64,1}}; permute=true, scale=true, sortby=LinearAlgebra.eigsortby)
Closest candidates are:
  eigen!(!Matched::SymTridiagonal{var"#s828",V} where V<:AbstractArray{var"#s828",1} where var"#s828"<:Union{Float32, Float64}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:279 got unsupported keyword arguments "permute", "scale", "sortby"
  eigen!(!Matched::SymTridiagonal{var"#s828",V} where V<:AbstractArray{var"#s828",1} where var"#s828"<:Union{Float32, Float64}, !Matched::UnitRange) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:282 got unsupported keyword arguments "permute", "scale", "sortby"
  eigen!(!Matched::SymTridiagonal{var"#s828",V} where V<:AbstractArray{var"#s828",1} where var"#s828"<:Union{Float32, Float64}, !Matched::Real, !Matched::Real) at /Users/julia/buildbot/worker/pa

If we're lucky, someone has implemented a generic solver in Julia that works for a wider range of types. Example:

In [48]:
Hbig = big.(H)
eigen(Hermitian(Hbig));

LoadError: MethodError: no method matching eigen!(::Hermitian{BigFloat,Array{BigFloat,2}}; sortby=nothing)
Closest candidates are:
  eigen!(!Matched::SymTridiagonal{var"#s828",V} where V<:AbstractArray{var"#s828",1} where var"#s828"<:Union{Float32, Float64}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:279 got unsupported keyword argument "sortby"
  eigen!(!Matched::SymTridiagonal{var"#s828",V} where V<:AbstractArray{var"#s828",1} where var"#s828"<:Union{Float32, Float64}, !Matched::UnitRange) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:282 got unsupported keyword argument "sortby"
  eigen!(!Matched::SymTridiagonal{var"#s828",V} where V<:AbstractArray{var"#s828",1} where var"#s828"<:Union{Float32, Float64}, !Matched::Real, !Matched::Real) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:287 got

In [49]:
using GenericLinearAlgebra

In [50]:
eigen(Hermitian(Hbig))

Eigen{BigFloat,BigFloat,Array{BigFloat,2},Array{BigFloat,1}}
values:
100-element Array{BigFloat,1}:
 -2.499032564583976129841491078128552885270844564275068999255406464586997523509518
 -2.49613119426718869664469376721353683820035347873096011215866399535318914672511
 -2.491298695938037160967995737868198811244191747682578616800626006817236297244357
 -2.484539744726553020184742526239320873977567777619982918612582442507556211531338
 -2.475860879481513441495886953659483104715504453725995724740209546532748607545501
 -2.465270496444527374874063414521910296283215789940930200782648270857681647241034
 -2.452778841127214059072142159451675184893454442233729896962151556967994660818164
 -2.438397998399332225887590822597361140711942686757388384968109952867283314193059
 -2.42214188079744909752949054940206867831399022597374035085438522526236180426978
 -2.404026215065459784760611736625730055990725724610455570939344743892098437920844
 -2.38406852693997826403007301588349407593352075137428043174518185249679

Arguably the most important matrix type in science applications is a sparse matrix, i.e. `SparseMatrixCSC`.

In [51]:
eigen(Hsparse)

LoadError: eigen(A) not supported for sparse matrices. Use for example eigs(A) from the Arpack package instead.

Let's follow Julia's advice and take a look at [ARPACK.jl](https://github.com/JuliaLinearAlgebra/Arpack.jl) and similar packages.

### Diagonalizing sparse matrices

[ARPACK.jl]() -  Wrapper to Fortran library [ARPACK](https://www.caam.rice.edu/software/ARPACK/) which implements **iterative** eigenvalue and singular value solvers. By far the most established sparse eigensolver.

Julia implementations:

* [ArnoldiMethod.jl](https://github.com/haampie/ArnoldiMethod.jl)
* [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl)
* [IterativeSolvers.jl](https://github.com/JuliaMath/IterativeSolvers.jl)
* and more


A key thing to remember is that while `eigen` is - up to numerical errors - exact, the methods in the packages above are iterative and approximative.

Arpack uses a different name for the eigenvalue decomposition. They called it `eigs`.

In [52]:
using Arpack
λ, evs = eigs(Hsparse);
λ

6-element Array{Float64,1}:
 -2.499032564583997
 -2.4961311942671984
 -2.491298695938046
 -2.484539744726571
 -2.4758608794815182
 -2.4652704964445267

In KrylovKit, they call the function `eigsolve`.

In [ ]:
using KrylovKit
λ, evs = eigsolve(Hsparse);
λ

# Core messages of this Notebook

* The standard libraries `LinearAlgebra` and `SparseArrays` make Julia speak linear algebra.
* **Indicate properties and structure of a matrix**, like hermiticity or sparsity, through types. Fallback to generic types only if you run into method errors.
* For **sparse matrix exact diagonalization**, ARPACK.jl is sort of a standard but there are great alternatives like KrylovKit.jl.